## Import libraries

In [1]:
import numpy as np
from numba import jit
import time

# The puzzle

In [2]:
sudoku_to_solve = [
    [0, 0, 0, 0, 0, 0, 2, 0, 0],
    [0, 8, 0, 0, 0, 7, 0, 9, 0],
    [6, 0, 2, 0, 0, 0, 5, 0, 0],
    [0, 7, 0, 0, 6, 0, 0, 0, 0],
    [0, 0, 0, 9, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 4, 0],
    [0, 0, 5, 0, 0, 0, 6, 0, 3],
    [0, 9, 0, 4, 0, 0, 0, 7, 0],
    [0, 0, 6, 0, 0, 0, 0, 0, 0]
    ]

## With JIT

In [3]:
@jit
def backtrack_jit(i,j,board):
    while board[i,j]!=0:                                  #While we are not at an empty space
        i, j = i + (j+1)//9, (j+1)%9                          #Increase position
        if i>=9:                                              #If we finish iterating the board
            return True                                           #Sudoku complete
    for test_num in range(1,10):                          #Test numbers 1-9
        if test_num in board[i,:]:                            #If it is in row
            continue                                              #Try next number
        if test_num in board[:,j]:                            #If it is in column
            continue                                               #Try next number
        if test_num in board[i-i%3:i-i%3+3,j-j%3:j-j%3+3]:    #If it is in sub square
            continue                                               #Try next number
        board[i,j] = test_num                                 #Assign the number
        if backtrack_jit(i,j,board):                          #if you can backtrack
            return True                                           #Sudoku complete
    board[i,j] = 0                                        #All numbers failed, make position empty
    return False                                              #Backtrack

### Compile time

In [4]:
start_time = time.time()
predef_arr = np.zeros((9,9)).astype(np.uint8)
backtrack_jit(0,0,predef_arr)
end_time = time.time()
print(end_time - start_time)
print(predef_arr)

0.8631448745727539
[[1 2 3 4 5 6 7 8 9]
 [4 5 6 7 8 9 1 2 3]
 [7 8 9 1 2 3 4 5 6]
 [2 1 4 3 6 5 8 9 7]
 [3 6 5 8 9 7 2 1 4]
 [8 9 7 2 1 4 3 6 5]
 [5 3 1 6 4 2 9 7 8]
 [6 4 2 9 7 8 5 3 1]
 [9 7 8 5 3 1 6 4 2]]


### Run time on zero initialization

In [5]:
start_time = time.time()
predef_arr = np.zeros((9,9)).astype(np.uint8)
backtrack_jit(0,0,predef_arr)
end_time = time.time()
print(end_time - start_time)
print(predef_arr)

0.0
[[1 2 3 4 5 6 7 8 9]
 [4 5 6 7 8 9 1 2 3]
 [7 8 9 1 2 3 4 5 6]
 [2 1 4 3 6 5 8 9 7]
 [3 6 5 8 9 7 2 1 4]
 [8 9 7 2 1 4 3 6 5]
 [5 3 1 6 4 2 9 7 8]
 [6 4 2 9 7 8 5 3 1]
 [9 7 8 5 3 1 6 4 2]]


### Run time on actual puzzle

In [6]:
start_time = time.time()
predef_arr = np.array(sudoku_to_solve).astype(np.uint8)
backtrack_jit(0,0,predef_arr)
end_time = time.time()
print(end_time - start_time)
print(predef_arr)

0.19260454177856445
[[9 5 7 6 1 3 2 8 4]
 [4 8 3 2 5 7 1 9 6]
 [6 1 2 8 4 9 5 3 7]
 [1 7 8 3 6 4 9 5 2]
 [5 2 4 9 7 1 3 6 8]
 [3 6 9 5 2 8 7 4 1]
 [8 4 5 7 9 2 6 1 3]
 [2 9 1 4 3 6 8 7 5]
 [7 3 6 1 8 5 4 2 9]]


## No JIT

In [7]:
def backtrack_py(i,j,board):
    while board[i,j]!=0:                                  #While we are not at an empty space
        i, j = i + (j+1)//9, (j+1)%9                          #Increase position
        if i>=9:                                              #If we finish iterating the board
            return True                                           #Sudoku complete
    for test_num in range(1,10):                          #Test numbers 1-9
        if test_num in board[i,:]:                            #If it is in row
            continue                                              #Try next number
        if test_num in board[:,j]:                            #If it is in column
            continue                                               #Try next number
        if test_num in board[i-i%3:i-i%3+3,j-j%3:j-j%3+3]:    #If it is in sub square
            continue                                               #Try next number
        board[i,j] = test_num                                 #Assign the number
        if backtrack_py(i,j,board):                           #if you can backtrack
            return True                                           #Sudoku complete
    board[i,j] = 0                                        #All numbers failed, make position empty
    return False                                              #Backtrack

### Run time on zero initialization

In [8]:
start_time = time.time()
predef_arr = np.zeros((9,9)).astype(np.uint8)
backtrack_py(0,0,predef_arr)
end_time = time.time()
print(end_time - start_time)
print(predef_arr)

0.01607680320739746
[[1 2 3 4 5 6 7 8 9]
 [4 5 6 7 8 9 1 2 3]
 [7 8 9 1 2 3 4 5 6]
 [2 1 4 3 6 5 8 9 7]
 [3 6 5 8 9 7 2 1 4]
 [8 9 7 2 1 4 3 6 5]
 [5 3 1 6 4 2 9 7 8]
 [6 4 2 9 7 8 5 3 1]
 [9 7 8 5 3 1 6 4 2]]


### Run time on actual puzzle

In [9]:
start_time = time.time()
predef_arr = np.array(sudoku_to_solve).astype(np.uint8)
backtrack_py(0,0,predef_arr)
end_time = time.time()
print(end_time - start_time)
print(predef_arr)

58.47101831436157
[[9 5 7 6 1 3 2 8 4]
 [4 8 3 2 5 7 1 9 6]
 [6 1 2 8 4 9 5 3 7]
 [1 7 8 3 6 4 9 5 2]
 [5 2 4 9 7 1 3 6 8]
 [3 6 9 5 2 8 7 4 1]
 [8 4 5 7 9 2 6 1 3]
 [2 9 1 4 3 6 8 7 5]
 [7 3 6 1 8 5 4 2 9]]
